# 1. Immobiliare Data Cleaning

In [0]:
import pandas as pd
import numpy as np
import dateparser

In [0]:
#Milan = spark.table("silver.edgar_sarto_revenue.immobiliare_v_3")
Milan = spark.table("silver.edgar_sarto_revenue.immobiliare_v_4")

Milan = Milan.toPandas()

Milan['City'] = 'MILAN'

## 1.1. City / Brough / Zone (Postal Code) / Agency

In [0]:
IMM = Milan.copy()

In [0]:
IMM.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2590 entries, 0 to 2589
Data columns (total 37 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Current_time  2590 non-null   datetime64[ns]
 1   Page_URL      2590 non-null   object        
 2   Text          2590 non-null   object        
 3   Text1         2590 non-null   object        
 4   Text10        2590 non-null   object        
 5   Text11        2590 non-null   object        
 6   Text12        2590 non-null   object        
 7   Text13        2590 non-null   object        
 8   Text14        2590 non-null   object        
 9   Text15        2590 non-null   object        
 10  Text16        2590 non-null   object        
 11  Text17        2590 non-null   object        
 12  Text18        2590 non-null   object        
 13  Text19        2590 non-null   object        
 14  Text2         2590 non-null   object        
 15  Text20        2590 non-null   object  

In [0]:
IMM.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2590 entries, 0 to 2589
Data columns (total 37 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Current_time  2590 non-null   datetime64[ns]
 1   Page_URL      2590 non-null   object        
 2   Text          2590 non-null   object        
 3   Text1         2590 non-null   object        
 4   Text10        2590 non-null   object        
 5   Text11        2590 non-null   object        
 6   Text12        2590 non-null   object        
 7   Text13        2590 non-null   object        
 8   Text14        2590 non-null   object        
 9   Text15        2590 non-null   object        
 10  Text16        2590 non-null   object        
 11  Text17        2590 non-null   object        
 12  Text18        2590 non-null   object        
 13  Text19        2590 non-null   object        
 14  Text2         2590 non-null   object        
 15  Text20        2590 non-null   object  

In [0]:
IMM = IMM.rename(columns={'Text1': 'Agency'})

IMM['Agency'] = IMM['Agency'].str.upper()

In [0]:
keywords = [
    'via', 'piazza', 'Via', 'Piazza', 'Magolfa , 21'
    ]

def delete_values(row):
    if pd.notna(row['Text7']) and any(keyword in row['Text7'] for keyword in keywords):
        row['Text7'], row['Text8'] = None, row['Text7']
    return row

IMM = IMM.apply(delete_values, axis=1)

IMM['District'] = IMM['Text7'].copy()

IMM.rename(columns={'Text7': 'Neighborhood', 'Text8': 'Street'}, inplace=True)

In [0]:
IMM['District'].unique()

array(['Paolo Sarpi', 'Solari', 'Centrale', 'Porta Venezia',
       'Vercelli - Wagner', 'Repubblica', 'Morgagni', 'Sempione',
       'Cenisio', 'Frua', 'Porta Nuova', 'Zara', 'Dezza', 'Isola',
       'Maggiolina', 'Farini', 'Buenos Aires', 'Washington',
       'Melchiorre Gioia', '', 'Navigli - Darsena', 'Dergano', 'Pasteur',
       None, 'Indipendenza', 'De Angeli', 'Arena',
       'Amendola - Buonarroti', 'Tripoli - Soderini', 'Palestro',
       'Quadronno - Crocetta', 'Moscova', 'Lanza', 'Corso Genova',
       'San Vittore', 'Ticinese', 'Turati', 'Guastalla',
       'Garibaldi - Corso Como', "Porta Romana - Medaglie d'Oro",
       'Quadrilatero della Moda', 'Brera', "Sant'Ambrogio", 'Bocconi',
       'Carrobbio', 'Corso San Gottardo', 'San Babila', 'Vincenzo Monti',
       'Montenero', 'Piave - Tricolore', 'Ascanio Sforza', 'Missori',
       'Cadorna - Castello', 'Borgogna - Largo Augusto'], dtype=object)

In [0]:
replace = {'Duomo': 'Centro','Brera': 'Centro','Scala - Manzoni': 'Centro','Missori': 'Centro',
           'San Babila':'Centro','Carrobbio': 'Centro','Lanza': 'Centro',"Sant'Ambrogio": 'Centro','Quadrilatero della Moda': 'Centro','Ticinese': 'Genova-Ticinese','Corso Genova': 'Genova-Ticinese','Navigli - Darsena': 'Genova-Ticinese','Corso San Gottardo': 'Genova-Ticinese','Quadronno - Crocetta': 'Quadronno-Palestro-Guastalla','Guastalla': 'Quadronno-Palestro-Guastalla','Palestro': 'Quadronno-Palestro-Guastalla','Borgogna - Largo Augusto': 'Quadronno-Palestro-Guastalla','Garibaldi - Corso Como': 'Garibaldi-Moscova-Porta Nuova','Moscova': 'Garibaldi-Moscova-Porta Nuova','Porta Nuova': 'Garibaldi-Moscova-Porta Nuova','Isola': 'Garibaldi-Moscova-Porta Nuova','Melchiorre Gioia': 'Garibaldi-Moscova-Porta Nuova','City Life': 'Fiera-Sempione-City Life-Portello','Pagano': 'Fiera-Sempione-City Life-Portello','Amendola - Buonarroti': 'Fiera-Sempione-City Life-Portello','Washington': 'Fiera-Sempione-City Life-Portello','Portello - Parco Vittoria': 'Fiera-Sempione-City Life-Portello','De Angeli': 'Fiera-Sempione-City Life-Portello','Monte Rosa - Lotto': 'Fiera-Sempione-City Life-Portello','Sempione': 'Fiera-Sempione-City Life-Portello','Arena': 'Fiera-Sempione-City Life-Portello','Farini': 'Fiera-Sempione-City Life-Portello','Navigli - Darsena': 'Navigli','Ripa di Porta Ticinese': 'Navigli','Corso San Gottardo': 'Navigli',"Porta Romana - Medaglie d'Oro": 'Porta Romana-Cadore-Montenero','Montenero': 'Porta Romana-Cadore-Montenero','Cadore': 'Porta Romana-Cadore-Montenero','Ripamonti': 'Porta Romana-Cadore-Montenero','Martini - Insubria': 'Porta Romana-Cadore-Montenero','Porta Vittoria': 'Porta Vittoria-Lodi','Lodi - Brenta': 'Porta Vittoria-Lodi','Famagosta': 'Famagosta-Barona','Solari': 'Solari-Washington','Washington': 'Solari-Washington','Dezza': 'Solari-Washington','Cenisio': 'Cenisio-Sarpi-Isola','Paolo Sarpi': 'Cenisio-Sarpi-Isola','Isola': 'Cenisio-Sarpi-Isola','Centrale': 'Centrale-Repubblica','Repubblica': 'Centrale-Repubblica','Turati': 'Centrale-Repubblica','Morgagni': 'Centrale-Repubblica','Porta Venezia': 'Porta Venezia-Indipendenza','Indipendenza': 'Porta Venezia-Indipendenza','Piave - Tricolore': 'Porta Venezia-Indipendenza','Città Studi': 'Porta Venezia-Indipendenza','Buenos Aires': 'Porta Venezia-Indipendenza','Vercelli - Wagner': 'Solari-Washington','Zara': 'Cenisio-Sarpi-Isola','Dergano': 'Cenisio-Sarpi-Isola','Frua': 'Solari-Washington','Pasteur': 'Porta Venezia-Indipendenza','Maggiolina': 'Cenisio-Sarpi-Isola','Bocconi': 'Navigli','Ascanio Sforza': 'Genova-Ticinese','Pezzotti - Meda': 'Porta Vittoria-Lodi','San Vittore': 'Centro','Plebisciti - Susa': 'Porta Venezia-Indipendenza','Vincenzo Monti': 'Fiera-Sempione-City Life-Portello','Cadorna - Castello': 'Centro','Arco della Pace': 'Fiera-Sempione-City Life-Portello'}
           

IMM['District'].replace(replace, regex=True, inplace=True)

keywords = [
    'Centro','Genova-Ticinese','Quadronno-Palestro-Guastalla','Garibaldi-Moscova-Porta Nuova','Fiera-Sempione-City Life-Portello','Navigli','Porta Romana-Cadore-Montenero','Porta Vittoria-Lodi','Famagosta-Barona','Solari-Washington','Cenisio-Sarpi-Isola','Centrale-Repubblica','Porta Venezia-Indipendenza']

def delete_values(row):
    if pd.notna(row['District']) and not any(keyword in row['District'] for keyword in keywords):
        row['District'] = None
    return row

IMM = IMM.apply(delete_values, axis=1)

IMM = IMM.drop(columns={'Text6', 'Text9'})

IMM['District'].value_counts()

Garibaldi-Moscova-Porta Nuova        385
Quadronno-Palestro-Guastalla         378
Genova-Ticinese                      353
Cenisio-Sarpi-Isola                  346
Solari-Washington                    314
Centrale-Repubblica                  267
Centro                               172
Porta Venezia-Indipendenza           140
Fiera-Sempione-City Life-Portello     90
Navigli                               44
Porta Romana-Cadore-Montenero         40
Name: District, dtype: int64

## 1.2. Prices

In [0]:
IMM['Text21'] = IMM['Text2'].copy()
IMM['Text31'] = IMM['Text3'].copy()
IMM['Text41'] = IMM['Text4'].copy()
IMM['Text51'] = IMM['Text5'].copy() 

def shift (row):
    if row['Text21'] is not None and ' €/m²' not in row['Text21']:
        row['Text21'],row['Text31'],row['Text41'],row['Text51'] = None, row['Text21'],row['Text31'],row['Text41']
    if row['Text31'] is not None and '/mese' not in row['Text31']:
        row['Text31'],row['Text41'],row['Text51'] = None, row['Text31'],row['Text41']
    if row['Text41'] is not None and '/mese' not in row['Text41']:
        row['Text41'],row['Text51'] = None, row['Text41']
    if row['Text41'] is not None and 'Non indicata' in row['Text41']:
        row['Text41'] = None
    return row

IMM = IMM.apply(shift, axis=1)

replace = {'€/m²':'', '€ ':'', '/mese':''}

IMM['Text21'].replace(replace, regex=True, inplace=True)
IMM['Text31'].replace(replace, regex=True, inplace=True)
IMM['Text41'].replace(replace, regex=True, inplace=True)

IMM['Text31'] = IMM['Text31'].str.replace('.', 'K')
IMM['Text41'] = IMM['Text41'].str.replace('.', 'K')

IMM['Text31'] = IMM['Text31'].str.replace('K', '')
IMM['Text41'] = IMM['Text41'].str.replace('K', '')

IMM = IMM.rename(columns={'Text21': '€_Sqm', 'Text31': 'Monthly_Price', 'Text41': 'Additional_Costs'})

IMM = IMM.drop(columns={'Text2', 'Text3', 'Text4', 'Text5'})


/home/spark-7dee7374-1841-4590-8adf-b3/.ipykernel/140187/command-5329508213600453-2188779280:25: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  IMM['Text31'] = IMM['Text31'].str.replace('.', 'K')
/home/spark-7dee7374-1841-4590-8adf-b3/.ipykernel/140187/command-5329508213600453-2188779280:26: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  IMM['Text41'] = IMM['Text41'].str.replace('.', 'K')


In [0]:
IMM.head(1)

,Current_time,Page_URL,Text,Agency,Text10,Text11,Text12,Text13,Text14,Text15,Text16,Text17,Text18,Text19,Text20,€_Sqm,Text22,Text23,Text24,Text25,Text26,Text27,Text28,Text29,Text30,Monthly_Price,Text32,Text33,Neighborhood,Street,City,District,Additional_Costs,Text51
0,2025-03-05 22:19:19.492,https://www.immobiliare.it/annunci/118747149/,"Milano\nPaolo Sarpi\nVia Francesco Londonio, 22",MILANO RESTYLE IMMOBILIARE DI MICHELE CECCHINI,1 locale,40 m²,1 bagno,Piano 3,Tipologia\nAttico | Intera proprietà | Classe ...,Piano\n3,Ascensore\nSì,Locali\n1,Cucina\nCucina a vista,Arredato\nSì,Terrazzo\nNo,25,"Contratto\nAffitto, 4+4",Piani edificio\n3,Superficie\n40 m²,Camere da letto\n1,Bagni\n1,Balcone\nNo,"Riscaldamento\nAutonomo, a radiatori, alimenta...",€ 990/mese,Prezzo\n€ 990/mese,990,Spese condominio\n€ 100/mese,Cauzione\n€ 2.970,Paolo Sarpi,Via Francesco Londonio,MILAN,Cenisio-Sarpi-Isola,100,€ 2.970


In [0]:
num_rows = len(IMM)

IMM['Balcony'] = [None] * num_rows
IMM['Terrace'] = [None] * num_rows
IMM['Heating_AC'] = [None] * num_rows
IMM['Parking'] = [None] * num_rows
IMM['Furnished'] = [None] * num_rows
IMM['Bedrooms'] = [None] * num_rows
IMM['Bathrooms'] = [None] * num_rows
IMM['Sqm2'] = [None] * num_rows
IMM['Elevator'] = [None] * num_rows
IMM['Floor'] = [None] * num_rows
IMM['Monthly_2'] = [None] * num_rows
IMM['€_Sqm_2'] = [None] * num_rows
IMM['Typology'] = [None] * num_rows

In [0]:
IMM['Text24'].unique()

array(['Superficie\n40 m²', 'Superficie\n75 m²', '', 'Superficie\n60 m²',
       'Superficie\n50 m²', 'Superficie\n55 m²', 'Superficie\n65 m²',
       'Superficie\n210 m²', 'Superficie\n150 m²', 'Superficie\n200 m²',
       'Superficie\n135 m²', 'Superficie\n63 m²', 'Superficie\n158 m²',
       'Superficie\n95 m²', 'Superficie\n72 m²', 'Superficie\n90 m²',
       'Superficie\n70 m²', 'Superficie\n45 m²', 'Superficie\n115 m²',
       'Superficie\n85 m²', 'Superficie\n68 m²', 'Superficie\n76 m²',
       'Superficie\n110 m²', 'Superficie\n83 m²', 'Superficie\n105 m²',
       'Superficie\n150 m² | commerciale 156 m²', 'Superficie\n102 m²',
       'Superficie\n78 m²', 'Superficie\n54 m² | commerciale 66 m²',
       'Superficie\n136 m²', 'Superficie\n215 m²', 'Superficie\n53 m²',
       'Superficie\n52 m²', 'Superficie\n59 m²', 'Superficie\n100 m²',
       'Locali\n2', 'Superficie\n134 m² | commerciale 136 m²',
       'Superficie\n80 m²', 'Superficie\n56 m²', 'Superficie\n48 m²',
       'Sup

In [0]:
import pandas as pd

columns = ['Text10', 'Text11', 'Text12', 'Text13', 'Text14', 'Text15', 
           'Text16', 'Text17', 'Text18', 'Text19', 'Text20', 'Text22', 
           'Text23', 'Text24', 'Text25', 'Text26', 'Text27', 'Text28']

def extract_features(row):
    features = {
        'Balcony': ['balcone', 'Balcone', 'balconi', 'Balconi'],
        'Terrace': ['terrazza', 'Terrazza', 'terrazzo', 'Terrazzo'],
        'Heating_AC': ['riscaldamento', 'Riscaldamento', 'Climatizzazione', 'climatizzazione'],
        'Parking': ['parcheggio', 'posti', 'box', 'garge', 'parking'],
        'Furnished': ['arredato', 'Arredato'],
        'Bedrooms': ['Locali', 'locali', 'Locale', 'locale'],
        'Bathrooms': ['Bagno', 'Bagni', 'bagno', 'bagni'],
        'Sqm2': [' m²', 'Superficie'],
        'Elevator': ['ascensore', 'Ascensore'],
        'Floor': ['piano', 'Piano'],
        'Monthly_2': ['/mese'],
        '€_Sqm_2': ['€/m²'],
        'Typology': ['tipologia', 'Tipologia']
    }
    
    for feature, keywords in features.items():
        match = next((row[column] for column in columns if pd.notna(row[column]) and any(keyword in row[column] for keyword in keywords)), None)
        if match:
            row[feature] = match
    return row

IMM = IMM.apply(extract_features, axis=1)

In [0]:
def balcony (row):
    if row['Balcony'] is not None and 'Sì' in row['Balcony']:
        row['Balcony'] = '1'
    if row['Balcony'] is not None and 'No' in row['Balcony']:
        row['Balcony'] = '0'
    if row['Balcony'] is not None and 'Balcone' in row['Balcony']:
        row['Balcony'] = '1'
    return row

IMM = IMM.apply(balcony, axis=1)

IMM['Balcony'].value_counts()

1    657
0    515
Name: Balcony, dtype: int64

In [0]:
def balcony (row):
    if row['Terrace'] is not None and 'Sì' in row['Terrace']:
        row['Terrace'] = '1'
    if row['Terrace'] is not None and 'No' in row['Terrace']:
        row['Terrace'] = '0'
    return row

IMM = IMM.apply(balcony, axis=1)

In [0]:
def balcony (row):
    if row['Heating_AC'] is not None:
        row['Heating_AC'] = '1'
    if row['Heating_AC'] is None:
        row['Heating_AC'] = '0'
    return row

IMM = IMM.apply(balcony, axis=1)

IMM['Heating_AC'].value_counts()

0    1502
1    1088
Name: Heating_AC, dtype: int64

In [0]:
def balcony (row):
    if row['Parking'] is not None:
        row['Parking'] = '1'
    if row['Parking'] is None:
        row['Parking'] = '0'
    return row

IMM = IMM.apply(balcony, axis=1)

IMM['Parking'].value_counts()

0    2443
1     147
Name: Parking, dtype: int64

In [0]:
def balcony (row):
    if row['Furnished'] is not None and 'Sì' in row['Furnished']:
        row['Furnished'] = '1'
    if row['Furnished'] is not None and 'Parzialmente Arredato' in row['Furnished']:
        row['Furnished'] = '1'
    else:
        row['Furnished'] = '0'
    return row

IMM = IMM.apply(balcony, axis=1)

IMM['Furnished'].value_counts()

0    2389
1     201
Name: Furnished, dtype: int64

In [0]:
replace = {' locali':'', ' locale':'', "\+ locali":'', " locali\n":'', 
           "3\+ bagni - uno o più adatto a persone disabili":''}

IMM['Bedrooms'].replace(replace, regex=True, inplace=True)

replace = {' bagno':'', ' bagni':'', "\+ bagni":''}

IMM['Bathrooms'].replace(replace, regex=True, inplace=True)


In [0]:
keywords = [
    'Ascensore', 
    'Ascensore\nSì',
    'Piano\nPiano terra, con ascensore, con accesso disabili',
    'Piano\n4°, con ascensore',
    'Piano\n6 piani: Interrato (-1), Piano terra, Piano rialzato, da 1° a 3°, con ascensore, con accesso disabili'
]

def elevator(row):
    if pd.notna(row['Elevator']) and any(keyword in row['Elevator'] for keyword in keywords):
        row['Elevator'] = '0'
    else:
        row['Elevator'] = '1'
    return row

IMM = IMM.apply(elevator, axis=1)

IMM['Elevator'].value_counts()

1    1326
0    1264
Name: Elevator, dtype: int64

In [0]:
import re

def extract_floor(floor_string):
    if pd.isna(floor_string):
        return None
    numbers = re.findall(r'\b\d+\b', floor_string)
    if numbers:
        return numbers[0]
    return None

IMM['Floor'] = [extract_floor(floor) for floor in IMM['Floor']]

In [0]:
def extract_typology(typology_string):
    if pd.isna(typology_string):
        return None
    parts = typology_string.split('\n', 1)
    return parts[1] if len(parts) > 1 else typology_string

IMM['Typology'] = [extract_typology(typology) for typology in IMM['Typology']]

IMM[['Typology_1', 'Typology_2', 'Typology_3']] = IMM['Typology'].str.split(' \| ', n=2, expand=True)

IMM['Typology_1'].unique()

array(['Attico', 'Appartamento', None, 'Loft', 'Mansarda', 'Open space',
       'Palazzo - Edificio', 'Villa unifamiliare',
       'Appartamento in villa'], dtype=object)

In [0]:
IMM['Sqm2'].replace(' m²', '', regex=True, inplace=True)

In [0]:
replace = {'Prezzo\n€ ': '', '/mese': ''}

IMM['Monthly_2'].replace(replace, regex=True, inplace=True)

In [0]:
IMM['Monthly_2'] = IMM['Monthly_2'].str.replace('.', 'K')
IMM['Monthly_2'] = IMM['Monthly_2'].str.replace('K', '')

/home/spark-7dee7374-1841-4590-8adf-b3/.ipykernel/140187/command-6684908947796194-441638092:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  IMM['Monthly_2'] = IMM['Monthly_2'].str.replace('.', 'K')


In [0]:
replace = {'Prezzo al m²\n': '', ' €/m²': ''}

IMM['€_Sqm_2'].replace(replace, regex=True, inplace=True)

In [0]:
IMM['€_Sqm_3'] = IMM['€_Sqm'].fillna(IMM['€_Sqm_2'])

IMM['Monthly_Price_3'] = IMM['Monthly_Price'].fillna(IMM['Monthly_2'])

In [0]:
IMM['Monthly_Price_3'].value_counts()

1500    109
2000     76
1300     54
2500     46
1600     45
       ... 
4190      1
5390      1
625       1
2215      1
1         1
Name: Monthly_Price_3, Length: 246, dtype: int64

In [0]:
IMM.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2590 entries, 0 to 2589
Data columns (total 52 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Current_time      2590 non-null   datetime64[ns]
 1   Page_URL          2590 non-null   object        
 2   Text              2590 non-null   object        
 3   Agency            2590 non-null   object        
 4   Text10            2590 non-null   object        
 5   Text11            2590 non-null   object        
 6   Text12            2590 non-null   object        
 7   Text13            2590 non-null   object        
 8   Text14            2590 non-null   object        
 9   Text15            2590 non-null   object        
 10  Text16            2590 non-null   object        
 11  Text17            2590 non-null   object        
 12  Text18            2590 non-null   object        
 13  Text19            2590 non-null   object        
 14  Text20            2590 n

In [0]:
IMM = IMM.drop(columns={'Text','Text10','Text11','Text12','Text13','Text14','Text15','Text16',
                        'Text17','Text18','Text19','Text20','€_Sqm','Text22','Text23','Text24','Text25','Text26','Text27','Text28','Monthly_Price','Text51','Sqm2','Monthly_2','€_Sqm_2','Typology'})

In [0]:
from geopy.geocoders import Nominatim
import time
geolocator = Nominatim(user_agent="my_custom_application_imm", timeout=16000)

In [0]:
data = [] 

Test_Geo = pd.DataFrame(data)

Test_Geo['Street'] = IMM['Street'].iloc[0:10000]

place = [Test_Geo['Street']]

In [0]:
from functools import lru_cache
import concurrent.futures

@lru_cache(maxsize=1000)
def geocode_address(address):
    try:
        location = geolocator.geocode(address, timeout=10)
        if location:
            return location.address
        else:
            return None
    except Exception as e:
        return None

def geocode_with_delay(address):
    result = geocode_address(address)
    time.sleep(1)
    return result

addresses = Test_Geo['Street'].tolist()

with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
    results = list(executor.map(geocode_with_delay, addresses))

Test_Geo['GeoAddress'] = results


In [0]:
Test_Geo_5 = Test_Geo

In [0]:
Test_Geo_IMM = [Test_Geo_5]

#Test_Geo_SPH = [Test_Geo_1, Test_Geo_2, Test_Geo_3, Test_Geo_4]

Test_Geo_IMM = pd.concat(Test_Geo_IMM, ignore_index=True)

In [0]:
IMM_Geo = pd.merge(IMM, Test_Geo_IMM, left_index=True, right_index=True)

In [0]:
def invertir_orden_columna(df, columna):
    df[columna] = df[columna].apply(lambda x: ", ".join(str(x).split(", ")[::-1]))
    return df

IMM_Geo = invertir_orden_columna(IMM_Geo, "GeoAddress")


In [0]:
IMM_Geo[['GeoAddress_1', 'GeoAddress_2', 'GeoAddress_3', 'GeoAddress_4', 'GeoAddress_5', 'GeoAddress_6',
         'GeoAddress_7','GeoAddress_8', 'GeoAddress_9', 'GeoAddress_10']] = IMM_Geo['GeoAddress'].str.split(',', n=9, expand=True)

In [0]:
def delete_unnecesary(row):
    if pd.notna(row['GeoAddress_1']) and ('Italia' not in row['GeoAddress_1'] and 'United Kingdom' not in row['GeoAddress_1']):
        row['GeoAddress_1'] = None
    return row

IMM_Geo = IMM_Geo.apply(delete_unnecesary, axis=1)

IMM_Geo['GeoAddress_1'].value_counts()

Italia    5707
Name: GeoAddress_1, dtype: int64

In [0]:
def delete_unnecesary(row):
    if row['GeoAddress_1'] is None:
        row['GeoAddress_2'] = None
        row['GeoAddress_3'] = None
        row['GeoAddress_4'] = None
    return row

IMM_Geo = IMM_Geo.apply(delete_unnecesary, axis=1)

In [0]:
def delete_unnecesary(row):
    if pd.notna(row['GeoAddress_1']) and ('Italia' not in row['GeoAddress_1'] and 'United Kingdom' not in row['GeoAddress_1']):
        row['GeoAddress_1'] = None
    return row

IMM_Geo = IMM_Geo.apply(delete_unnecesary, axis=1)

In [0]:
def delete_unnecesary(row):
    if pd.notna(row['GeoAddress_2']) and any(x in row['GeoAddress_2'] for x in ['Lazio', 'Lombardia', 'England']):
        row['GeoAddress_3'], row['GeoAddress_4'], row['GeoAddress_5'] = row['GeoAddress_2'],row['GeoAddress_3'], row['GeoAddress_4']
    return row

IMM_Geo = IMM_Geo.apply(delete_unnecesary, axis=1)

def delete_unnecesary(row):
    if pd.notna(row['GeoAddress_3']) and not any(x in row['GeoAddress_3'] for x in ['Lazio', 'Lombardia', 'England']):
        row['GeoAddress_3'], row['GeoAddress_4'], row['GeoAddress_5'] = None, row['GeoAddress_3'], row['GeoAddress_4']
    return row

IMM_Geo = IMM_Geo.apply(delete_unnecesary, axis=1)

In [0]:
IMM_Geo = IMM_Geo.drop(columns={'GeoAddress_4', 'GeoAddress_5', 'GeoAddress_6', 'GeoAddress_7', 'GeoAddress_8', 'GeoAddress_9', 'GeoAddress_10'})
IMM_Geo = IMM_Geo.rename(columns={'GeoAddress_1': 'Country','GeoAddress_2': 'Posal_Code', 'GeoAddress_3': 'Region'})

In [0]:
def delete_unnecesary(row):
    if pd.notna(row['Posal_Code']) and any(x in row['Posal_Code'] for x in [' Northern Ireland / Tuaisceart Éireann', ' Toscana', ' Lombardia', 
                                                                            ' England', ' Piemonte']):
        row['Posal_Code'] = None
    return row

IMM_Geo = IMM_Geo.apply(delete_unnecesary, axis=1)

In [0]:
IMM_Geo_Spark = spark.createDataFrame(IMM_Geo)

In [0]:
IMM_Geo_Spark.write.mode("overwrite").saveAsTable("silver.edgar_sarto_revenue.Immobiliarie_MILAN_Clean_2")